In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import string
random.seed(0)

def random_string(length=24):
    return ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(length))

def random_name():
    first_names = ['Michael', 'Crystal', 'Mike', 'Albertine', 'Cecil', 'Kathleen']
    middle_names = ['A', 'B', 'C', 'D', 'E', 'F']
    last_names = ['Mooney', 'Boyer', 'Test', 'Real']
    return random.choice(first_names), random.choice(middle_names), random.choice(last_names)

def random_datetime_generator(date):
    random_time = datetime.combine(date, datetime.min.time()) + timedelta(seconds=random.randint(0, 86399))
    return random_time

def create_acctsumm_data(num_rows=10):
    global depositor_ids 
    depositor_ids = [random_string(12) for _ in range(num_rows)]  # Generate DepositorIDs
    data_df = {
        'DepositorID': depositor_ids,
        'FirstName': [random_name()[0] for _ in range(num_rows)],
        'MiddleName': [random_name()[1] for _ in range(num_rows)],
        'LastName': [random_name()[2] for _ in range(num_rows)],
        'Status': np.random.choice(['Active', 'Inactive', 'Closed'], num_rows),
        'VirtualAccountNumber': ['10000000' + ''.join(random.choice(string.digits) for _ in range(2)) for _ in range(num_rows)],
        'OpenDate': [(datetime.now() - timedelta(days=random.randint(0,365))).strftime('%m/%d/%Y %H:%M') for _ in range(num_rows)],
        'PendingBalance': np.random.uniform(0, 1000, num_rows).round(2),
        'AvailableBalance': np.random.uniform(0, 20000, num_rows).round(2),
        'FirstDeposit': [(datetime.now() - timedelta(days=random.randint(0,365))).strftime('%Y-%m-%d %H:%M:%S.%f') for _ in range(num_rows)],
    }
    return pd.DataFrame(data_df)

def create_transsumm_data(num_rows=10):
    dates = [datetime.now() - timedelta(days=random.randint(0,90)) for i in range(num_rows)]
    random_dates = [random_datetime_generator(date) for date in dates]
    
    data_df1 = {
        'Date': random_dates,
        'TransactionID': [random_string() for _ in range(num_rows)],
        'DepositAmount': np.random.choice([0, 500, 1000, 2500, 5000, 10000, 15000], num_rows),
        'WithdrawalAmount': np.random.choice([0, 100, 500, 1000, 2500, 5000, 7500], num_rows),
        'DepositorID': random.choices(depositor_ids, k=num_rows), 
    }
    df1 = pd.DataFrame(data_df1)
    df1['Date'] = df1['Date'].dt.tz_localize('UTC')
    df1['Date'] = df1['Date'].dt.tz_convert('America/New_York')
    return df1

# Generate the DataFrames
acctsumm_df=create_acctsumm_data(num_rows=10)
transsumm_df= create_transsumm_data(num_rows=10000)


display(transsumm_df.head(), acctsumm_df.head())


,Date,TransactionID,DepositAmount,WithdrawalAmount,DepositorID
0,2023-12-20 03:42:00-05:00,c5qigR9gws2EPy5qyZ38oyNm,2500,7500,siWgNZq6ITZM
1,2024-01-14 21:08:16-05:00,XbpggxKom7RJPmvjqeOPJ3nc,500,1000,5jtgUe52RvEJ
2,2023-12-02 03:01:36-05:00,AXTAcczlYgU8zpSPVQuzF48s,500,2500,3HqdZ6J6afU1
3,2024-01-09 11:06:31-05:00,2XUJMHE5XOmBsTSobrd5YTp7,5000,1000,5jtgUe52RvEJ
4,2024-01-04 19:23:50-05:00,TcQIaV3wW5abg3M5ZC3jrlnB,10000,2500,9jZICffu4G7F


,DepositorID,FirstName,MiddleName,LastName,Status,VirtualAccountNumber,OpenDate,PendingBalance,AvailableBalance,FirstDeposit
0,2yW4Acq9GFz6,Cecil,E,Mooney,Closed,1000000013,03/03/2023 01:07,278.25,866.46,2023-12-25 01:07:43.805814
1,Y1t9EwL56nGi,Albertine,C,Mooney,Closed,1000000014,03/16/2023 01:07,490.20,13289.09,2023-04-24 01:07:43.805814
2,siWgNZq6ITZM,Mike,F,Boyer,Closed,1000000056,11/11/2023 01:07,799.00,14827.95,2023-07-12 01:07:43.805814
3,5jtgUe52RvEJ,Mike,C,Mooney,Closed,1000000020,01/25/2024 01:07,851.34,17598.64,2023-03-20 01:07:43.805814
4,gwBuNO6n9JEC,Crystal,E,Test,Closed,1000000087,03/14/2023 01:07,339.02,1683.71,2023-11-26 01:07:43.805814


In [20]:
from pytz import timezone
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

# custom business day offset that accounts for US federal holidays
us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())

def get_dates(df, hour =10):
    # hour = 10 means a transaction done after 8 pm will be processed at 10 am the next BUSINESS day

    cutoff_hour = 20 
    et_zone = timezone('US/Eastern')
    
    def get_next_business_day(transaction_time):
        if transaction_time.hour>=cutoff_hour:
            return transaction_time.normalize() + timedelta(days=0, hours=hour) + 1* us_bd
        else:
            return transaction_time
    
    df['TransactionTimeET'] = df['Date'].dt.tz_convert(et_zone)
    df['ProcessingDate'] = df['TransactionTimeET'].apply(get_next_business_day)
    df['WithdrawableDate'] = df['ProcessingDate'] + 5* us_bd ########################## t+5 BUSINESS day hold 
    df['IntrestCalulationDate'] = df['ProcessingDate'] + timedelta(days=1)############# t+1 NORMAL day hold

    current_date_et = datetime.now(et_zone)
    df['WithdrawableFlag'] = df['WithdrawableDate'].apply(lambda x: current_date_et >= x)
    return df.drop(columns=['TransactionTimeET'])

transsumm_df_with_dates = get_dates(transsumm_df)
transsumm_df_with_dates.head()


c:\Users\RishiRaj\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arrays\datetimes.py:760: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex.
  warnings.warn(


,Date,TransactionID,DepositAmount,WithdrawalAmount,DepositorID,ProcessingDate,WithdrawableDate,IntrestCalulationDate,WithdrawableFlag
0,2023-12-20 03:42:00-05:00,c5qigR9gws2EPy5qyZ38oyNm,2500,7500,siWgNZq6ITZM,2023-12-20 03:42:00-05:00,2023-12-28 03:42:00-05:00,2023-12-21 03:42:00-05:00,True
1,2024-01-14 21:08:16-05:00,XbpggxKom7RJPmvjqeOPJ3nc,500,1000,5jtgUe52RvEJ,2024-01-16 10:00:00-05:00,2024-01-23 10:00:00-05:00,2024-01-17 10:00:00-05:00,True
2,2023-12-02 03:01:36-05:00,AXTAcczlYgU8zpSPVQuzF48s,500,2500,3HqdZ6J6afU1,2023-12-02 03:01:36-05:00,2023-12-08 03:01:36-05:00,2023-12-03 03:01:36-05:00,True
3,2024-01-09 11:06:31-05:00,2XUJMHE5XOmBsTSobrd5YTp7,5000,1000,5jtgUe52RvEJ,2024-01-09 11:06:31-05:00,2024-01-17 11:06:31-05:00,2024-01-10 11:06:31-05:00,True
4,2024-01-04 19:23:50-05:00,TcQIaV3wW5abg3M5ZC3jrlnB,10000,2500,9jZICffu4G7F,2024-01-04 19:23:50-05:00,2024-01-11 19:23:50-05:00,2024-01-05 19:23:50-05:00,True


In [21]:
x = datetime.strptime("2024-02-16 15:30:00", "%Y-%m-%d %H:%M:%S")
print(x)
x+ us_bd*1  # accounts for 19 feb holiday

2024-02-16 15:30:00


Timestamp('2024-02-20 15:30:00')

In [22]:
# transsumm_df_with_dates['ProcessingDate'] = transsumm_df_with_dates['ProcessingDate'].dt.normalize()
# processed_deposits_by_day = transsumm_df_with_dates.groupby(['ProcessingDate', 'DepositorID'])['DepositAmount'].sum().reset_index()
# processed_deposits_by_day.head()

In [23]:
transsumm_df_with_dates['IntrestCalulationDate'] = transsumm_df_with_dates['IntrestCalulationDate'].dt.normalize()
deposits_by_day = transsumm_df_with_dates.groupby(['IntrestCalulationDate', 'DepositorID'])['DepositAmount'].sum().reset_index()
deposits_by_day.head()

,IntrestCalulationDate,DepositorID,DepositAmount
0,2023-11-25 00:00:00-05:00,2yW4Acq9GFz6,500
1,2023-11-25 00:00:00-05:00,3HqdZ6J6afU1,6000
2,2023-11-25 00:00:00-05:00,Y1t9EwL56nGi,5000
3,2023-11-25 00:00:00-05:00,gwBuNO6n9JEC,2500
4,2023-11-25 00:00:00-05:00,siWgNZq6ITZM,5000


In [26]:
def is_leap_year(year):
    return year % 400 == 0 or (year % 100 != 0 and year % 4 == 0)

deposits_by_day['DaysInYear'] = deposits_by_day['IntrestCalulationDate'].apply(lambda x: 366 if is_leap_year(x.year) else 365)
apy = 5.36 / 100
deposits_by_day['DailyInterestRate'] = apy / deposits_by_day['DaysInYear'] ######## ignoring the compounding effects of APY for simplicity

current_date = datetime.now().date()
deposits_by_day['DaysHeld'] = (current_date - deposits_by_day['IntrestCalulationDate'].dt.date).apply(lambda x: x.days+1)
deposits_by_day['Interest'] = deposits_by_day['DepositAmount'] * deposits_by_day['DailyInterestRate'] * deposits_by_day['DaysHeld']

deposits_by_day

,IntrestCalulationDate,DepositorID,DepositAmount,DaysInYear,DailyInterestRate,DaysHeld,Interest
0,2023-11-25 00:00:00-05:00,2yW4Acq9GFz6,500,365,0.000147,91,6.681644
1,2023-11-25 00:00:00-05:00,3HqdZ6J6afU1,6000,365,0.000147,91,80.179726
2,2023-11-25 00:00:00-05:00,Y1t9EwL56nGi,5000,365,0.000147,91,66.816438
3,2023-11-25 00:00:00-05:00,gwBuNO6n9JEC,2500,365,0.000147,91,33.408219
4,2023-11-25 00:00:00-05:00,siWgNZq6ITZM,5000,365,0.000147,91,66.816438
...,...,...,...,...,...,...,...
911,2024-02-24 00:00:00-05:00,gtJsThJv07In,65500,366,0.000146,0,0.000000
912,2024-02-24 00:00:00-05:00,gwBuNO6n9JEC,41500,366,0.000146,0,0.000000
913,2024-02-24 00:00:00-05:00,pUuT3em6KopZ,55000,366,0.000146,0,0.000000
914,2024-02-24 00:00:00-05:00,siWgNZq6ITZM,30000,366,0.000146,0,0.000000


In [29]:
amount = deposits_by_day.groupby(['DepositorID'])[['DepositAmount','Interest']].sum().reset_index()
amount

,DepositorID,DepositAmount,Interest
0,2yW4Acq9GFz6,4509000,30162.431035
1,3HqdZ6J6afU1,4883500,32045.841509
2,5jtgUe52RvEJ,4723000,30719.618864
3,9jZICffu4G7F,4788500,30897.166306
4,Y1t9EwL56nGi,4706500,29816.757778
5,gtJsThJv07In,5103000,32237.975783
6,gwBuNO6n9JEC,5049000,32666.047354
7,pUuT3em6KopZ,4648500,29954.187071
8,siWgNZq6ITZM,4764000,33276.008252
9,zT0YQOaNF03v,4678000,31586.983340


In [ ]:
from datetime import timedelta
import pandas as pd

class tests:
    def __init__(self, run_id):
        self.run_id = run_id
    
    def test_A(self):
        print('==========Test A==========')
        # ... Test A implementation ...
        pass
    
    def test_B2(self):
        print('==========Test B2==========')
        # Assuming transactions_df is a DataFrame containing all transactions with columns 'TransactionDate', 'IsReturned', and 'DepositAmount'.
        transactions_df = self.get_transactions()  # You would have to implement this method
        business_days_offset = 1
        
        # Adjust the Interest Calculation Date to +1 business day after deposit
        transactions_df['InterestCalculationStartDate'] = transactions_df['TransactionDate'].apply(
            lambda x: self.get_next_business_day(x, business_days_offset)
        )
        
        # Filter out returned ACH transactions
        transactions_df = transactions_df[~transactions_df['IsReturned']]
        
        # Here you would implement the interest calculation logic,
        # but I will leave it with a placeholder as it depends on your specific logic and data structure.
        # ...
        # Assume we have a function to calculate interest
        transactions_df['Interest'] = transactions_df.apply(lambda row: self.calculate_interest(row), axis=1)
        
        # Output the result for verification
        print(transactions_df.head())  # Or any other form of verification or assertion as needed
        pass

    def get_next_business_day(self, current_date, offset):
        next_business_day = current_date
        while offset > 0:
            next_business_day += timedelta(days=1)
            # Skip weekends
            if next_business_day.weekday() < 5:  # Monday to Friday are business days (0-4)
                offset -= 1
        return next_business_day

    def get_transactions(self):
        # This method should return the DataFrame with your transactions.
        # For this example, it's just a placeholder.
        # Replace with the actual code to retrieve your transaction data.
        # Here's a mock structure for illustration purposes:
        return pd.DataFrame({
            'TransactionDate': [pd.Timestamp('2024-03-01'), pd.Timestamp('2024-03-02')],
            'IsReturned': [False, True],
            'DepositAmount': [1000, 2000]
        })

    def calculate_interest(self, transaction_row):
        # Placeholder for interest calculation logic
        # Replace with actual calculation based on your interest rules.
        return transaction_row['DepositAmount'] * 0.05  # Example fixed interest for demonstration

# To run the test:
test_instance = tests(run_id='123')
test_instance.test_B2()
